In order to determine the correct values of $h$, $\Omega_m$, and $\Omega_b$ for nonzero values of $w_a$, the values of $h^2 \Omega_m$, $h^2 \Omega_b$, and $D_* \approx D_A(z=1091)$ are to remain constant. Base values from the flat $\Lambda$CDM model from [1] are used. In order to avoid unphysical perturbations if $w(z) = -1.0$ for any $z$ when not modelled as a cosmological constant, $w(z)$ choices of $w_0 = -0.99$, $w_a = 0.5$ and $w_0 = -1.01$, $w_a = -0.5$ were made.

Refer back to arxiv link:
\[[1](http://arxiv.org/abs/1201.2434v2)]

In [1]:
%pylab inline
import scipy.integrate
from astropy import cosmology, units as u, constants as const
from classy import Class

Populating the interactive namespace from numpy and matplotlib


In [143]:
Z_lss = 1091.0
cosmo1 = cosmology.FlatLambdaCDM(71.0, 0.222 + 0.045, Ob0 = 0.045)

In [139]:
def get_class_parameters(cosmology):
    """
    Get CLASS parameters corresponding to an astropy cosmology model.
    """
    class_parameters = {}
    try:
        class_parameters['h'] = cosmology.h
        class_parameters['T_cmb'] = cosmology.Tcmb0.value
        class_parameters['Omega_b'] = cosmology.Ob0
        # CDM = DM - massive neutrinos.
        class_parameters['Omega_cdm'] = cosmology.Om0 - cosmology.Ob0
        class_parameters['Omega_k'] = cosmology.Ok0
        # Dark energy sector. CLASS will calculate whichever of
        # Omega_Lambda/fld/scf we do not provide.
        class_parameters['Omega_scf'] = 0.
        try:
            # Only subclasses of wCDM have the w0 attribute.
            class_parameters['w0_fld'] = cosmology.w0
            class_parameters['wa_fld'] = cosmology.wa
            class_parameters['Omega_Lambda'] = 0.
        except AttributeError:
            class_parameters['Omega_fld'] = 0.
        # Neutrino sector.
        if cosmology.has_massive_nu:
            m_nu = cosmology.m_nu
            try:
                num_massive = len(m_nu)
            except TypeError:
                num_massive = 3
                m_nu = [num_massive * m_nu]
            if num_massive > 3:
                raise RuntimeError(
                    'Cannot calculate cosmology with >3 massive neutrinos.')
            m_ncdm = []
            for mass in m_nu:
                if mass > 0:
                    m_ncdm.append(repr(mass.value))
                else:
                    num_massive -= 1
            if num_massive > 0:
                class_parameters['m_ncdm'] = ','.join(m_ncdm)
        else:
            num_massive = 0
        class_parameters['N_ncdm'] = num_massive
        num_light = 3 - num_massive
        class_parameters['N_ur'] = (3.00641 - num_massive +
            num_light*(cosmology.Neff - 3.00641)/3.)
    except AttributeError:
        raise ValueError('Cosmology is missing required attributes.')
    return class_parameters

In [96]:
cosmo = Class()

In [98]:
cosmo.struct_cleanup()
cosmo.empty()

In [144]:
print('Base numbers:')
cosmo.set(get_class_parameters(cosmo1))
cosmo.compute()
print(cosmo.omega_b())
print(cosmo.Omega_m() * cosmo.h() ** 2)
dist = cosmo.angular_distance(Z_lss)
print(dist)
cosmo.struct_cleanup()
cosmo.empty()

Base numbers:
0.0226845
0.1345947
12.8764593039


In [132]:
h = np.linspace(0.6,0.8,num=201)
b = np.around(0.0226845 * h ** -2, decimals = 3)
m = np.around(0.1345947 * h ** -2, decimals = 3)
c = m - b

In [130]:
def params_pos():
    test = {}
    test['N_ncdm'] = 0.0
    test['N_ur'] = 3.04
    test['Omega_Lambda'] = 0.0
    test['Omega_scf'] = 0.0
    test['Omega_k'] = 0.0
    test['w0_fld'] = -1.0 + 0.01
    test['wa_fld'] = 0.5
    test['T_cmb'] = 2.725
    return test
def params_neg():
    test = {}
    test['N_ncdm'] = 0.0
    test['N_ur'] = 3.04
    test['Omega_Lambda'] = 0.0
    test['Omega_scf'] = 0.0
    test['Omega_k'] = 0.0
    test['w0_fld'] = -1.0 - 0.01
    test['wa_fld'] = -0.5
    test['T_cmb'] = 2.725
    return test

In [145]:
trial = []
print('w0 = -0.99, wa = 0.5:')
for i in range(201):
    test = params_pos()
    test['h'] = h[i]
    test['Omega_b'] = b[i]
    test['Omega_cdm'] = c[i]
    cosmo.set(test)
    cosmo.compute()
    trial.append(np.abs(dist - cosmo.angular_distance(Z_lss)))
    cosmo.struct_cleanup()
    cosmo.empty()
j = np.argmin(trial)
print("h = {}, Om = {}, Ob = {}".format(h[j],m[j],b[j]))

w0 = -0.99, wa = 0.5:
h = 0.704, Om = 0.272, Ob = 0.046


In [146]:
trial = []
print('w0 = -1.01, wa = -0.5:')
for i in range(201):
    test = params_neg()
    test['h'] = h[i]
    test['Omega_b'] = b[i]
    test['Omega_cdm'] = c[i]
    cosmo.set(test)
    cosmo.compute()
    trial.append(np.abs(dist - cosmo.angular_distance(Z_lss)))
    cosmo.struct_cleanup()
    cosmo.empty()
k = np.argmin(trial)
print("h = {}, Om = {}, Ob = {}".format(h[k],m[k],b[k]))

w0 = -1.01, wa = -0.5:
h = 0.715, Om = 0.263, Ob = 0.044
